In [3]:
from __future__ import print_function
!git clone https://github.com/kundajelab/revcomp_experiments.git
%cd revcomp_experiments/
!python setup.py install
%cd ..

fatal: destination path 'revcomp_experiments' already exists and is not an empty directory.
/mnt/lab_data2/hannahgz/revcomp_experiments/revcomp_experiments/revcomp_experiments
running install
running bdist_egg
running egg_info
writing revcompexp.egg-info/PKG-INFO
writing dependency_links to revcompexp.egg-info/dependency_links.txt
writing requirements to revcompexp.egg-info/requires.txt
writing top-level names to revcompexp.egg-info/top_level.txt
reading manifest file 'revcompexp.egg-info/SOURCES.txt'
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/bdist.linux-x86_64/egg/EGG-INFO/scripts
copying build/scripts-3.7/motif_density_and_position_sim.py -> build/bdist.linux-x86_64/egg/EGG-INFO/scrip

In [4]:
!pip install concise
!pip install keras-genomics

In [5]:
!pip install git+https://github.com/kundajelab/simdna.git@v0.4.3.1#egg=simdna
!git clone https://github.com/kundajelab/revcomp_experiments.git
%cd revcomp_experiments/
!python setup.py install
%cd ..
import simdna

fatal: destination path 'revcomp_experiments' already exists and is not an empty directory.
/mnt/lab_data2/hannahgz/revcomp_experiments/revcomp_experiments/revcomp_experiments
running install
running bdist_egg
running egg_info
writing revcompexp.egg-info/PKG-INFO
writing dependency_links to revcompexp.egg-info/dependency_links.txt
writing requirements to revcompexp.egg-info/requires.txt
writing top-level names to revcompexp.egg-info/top_level.txt
reading manifest file 'revcompexp.egg-info/SOURCES.txt'
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/bdist.linux-x86_64/egg/EGG-INFO/scripts
copying build/scripts-3.7/motif_density_and_position_sim.py -> build/bdist.linux-x86_64/egg/EGG-INFO/scrip

In [6]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [7]:
!motif_density_and_position_sim.py --seed 1234 --motifNames GATA_disc1 TAL1_known1 --max-motifs 4 --min-motifs 1 --mean-motifs 2 --zero-prob 0.5 --seqLength 1000 --posSdevInBp 100 --numSeqs 10000

In [8]:
!head DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata

seqName	sequence	embeddings
synth0	AGCTTAATTTCGGTCGGATTCGACTGCTCGTCTATTATCTAAAGGGAGCGAGGAGTTTTTAGCACACTAATGCAATCGACCGTGTATTGCAGGTCGGTAGTTTGTAAGATCACTTACATTTTTGAACGAACGTAAATACGATTGACTAGATGAATTTTGGAGTTTTGCTTGCCTATCTTTGCTTCACTCCGAATACGATTTTGTTTACACATGGGCCCCTCATTCGCAACAGTTCACTTCAAAGAAAGATCTGAGGATGGGGATTTGCTGCGCAGTTGACAAGAGGTTTAAGTTGCGTGAATCGGAAGTCTAAAAAGTTGTGGGTATAAAAGCTACTCTGTTTATGATAATTGCAGTTTAGCCATGATTCTGTTTACCATCTGTACACTGTAACTTCTACTTAAGTCAGGATTTATTTTACTTGTGATCTCATAGAAGTGTCAATACAGGGTTATGATGGCCTAACAGATGTAATAAACGATATCTCACTAGCGAATTAGTTGAATAGTAGCAAAAACATTTAGTTACGGGTTCATCATAGCAACAGATGGCCGAATTCAAATACGGCCACTTATATAACTTCAGAAGGATTTCCCCGGATTATAGAACACTGGGAGTAGTTCCTAAACAGACCTGGGATCAGATGGTCGACTTCAGCTGTTCAACGGGATTCTAAGTAGAAATCAGGCTACACACGAAATCCTACGTCTGTTTAAAATTTCGAATATTCTCAGCGCGTGTAAGGTTTGATCGGCACTGAGATTGCCTAGCAATTTCAGGTGATTATATTTGCGATACTCGATGCATAAAGGTATACTAATTAGCTAAACAAAATATCTACGGCCCCCCGTGCATAGTTCATGTGCTTGTATTCTGCGTACTGCACTGCGATGTTCTTGGTCATGCATGACGCCCGTTTGGTGTTTGACAATAAGGCCAAAATCTTTTGCTAACCTGAGTATATG

In [9]:
#@title
from __future__ import absolute_import, division, print_function
from collections import OrderedDict
import re
import gzip
import os
import json
from simdna import random


DEFAULT_LETTER_TO_INDEX = {'A': 0, 'C': 1, 'G': 2, 'T': 3}


DEFAULT_BACKGROUND_FREQ = OrderedDict(
    [('A', 0.3), ('C', 0.2), ('G', 0.2), ('T', 0.3)])


DEFAULT_DINUC_FREQ = OrderedDict([
 ('AA',0.095),
 ('AC',0.050),
 ('AG',0.071),
 ('AT',0.075),
 ('CA',0.073),
 ('CC',0.054),
 ('CG',0.010),
 ('CT',0.072),
 ('GA',0.060),
 ('GC',0.044),
 ('GG',0.054),
 ('GT',0.050),
 ('TA',0.064),
 ('TC',0.060),
 ('TG',0.073),
 ('TT',0.095),
])


def get_file_handle(filename,mode="r"):
    if (re.search('.gz$',filename) or re.search('.gzip',filename)):
        if (mode=="r"):
            mode="rb";
        elif (mode=="w"):
            #I think write will actually append if the file already
            #exists...so you want to remove it if it exists
            if os.path.isfile(filename):
                os.remove(filename);
        return gzip.open(filename,mode)
    else:
        return open(filename,mode) 


def default_tab_seppd(s):
    s = trim_newline(s)
    s = split_by_delimiter(s, "\t")
    return s


def trim_newline(s):
    return s.rstrip('\r\n')


def split_by_delimiter(s, delimiter):
    return s.split(delimiter)


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
        line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();


def process_line(line, i, ignore_input_title,
                 transformation, action, progress_update=None):
    if (i > 1 or (ignore_input_title==False)):
        action(transformation(line),i)


def print_progress(progress_update, i, file_name=None):
    if progress_update is not None:
        if (i%progress_update == 0):
            print ("Processed "+str(i)+" lines"
                   +str("" if file_name is None else " of "+file_name))


class VariableWrapper():
    """ For when I want reference-type access to an immutable"""
    def __init__(self, var):
        self.var = var   


def enum(**enums):
    class Enum(object):
        pass
    to_return = Enum
    for key,val in enums.items():
        if hasattr(val, '__call__'): 
            setattr(to_return, key, staticmethod(val))
        else:
            setattr(to_return, key, val)
    to_return.vals = [x for x in enums.values()]
    to_return.the_dict = enums
    return to_return


def combine_enums(*enums):
    new_enum_dict = OrderedDict()
    for an_enum in enums:
        new_enum_dict.update(an_enum.the_dict)
    return enum(**new_enum_dict)


def sampleFromProbsArr(arrWithProbs):
    """Samples from a discrete distribution.
    Arguments:
        arrWithProbs: array of probabilities
    Returns:
        an index, sampled with the probability of that index in
    array of probabilities.
    """
    randNum = random.random()
    cdfSoFar = 0
    for (idx, prob) in enumerate(arrWithProbs):
        cdfSoFar += prob
        if (cdfSoFar >= randNum or idx == (len(arrWithProbs) - 1)):  # need the
            # letterIdx==(len(row)-1) clause because of potential floating point errors
            # that mean arrWithProbs doesn't sum to 1
            return idx


reverseComplementLookup = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G',
                           'a': 't', 't': 'a', 'g': 'c', 'c': 'g', 'N': 'N', 'n': 'n'}


def reverseComplement(sequence):
    reversedSequence = sequence[::-1]
    reverseComplemented = "".join(
        [reverseComplementLookup[x] for x in reversedSequence])
    return reverseComplemented


def sampleWithoutReplacement(arr, numToSample):
    arrayCopy = [x for x in arr]
    for i in range(numToSample):
        randomIndex = int(random.random() * (len(arrayCopy) - i)) + i
        swapIndices(arrayCopy, i, randomIndex)
    return arrayCopy[0:numToSample]


def swapIndices(arr, idx1, idx2):
    temp = arr[idx1]
    arr[idx1] = arr[idx2]
    arr[idx2] = temp


def get_file_name_parts(file_name):
    p = re.compile(r"^(.*/)?([^\./]+)(\.[^/]*)?$")
    m = p.search(file_name)
    return FileNameParts(m.group(1), m.group(2), m.group(3))


class FileNameParts(object):

    def __init__(self, directory, core_file_name, extension):
        self.directory = directory if (directory is not None) else os.getcwd()
        self.core_file_name = core_file_name
        self.extension = extension

    def get_full_path(self):
        return self.directory+"/"+self.file_name

    def get_core_file_name_and_extension(self):
        return self.core_file_name+self.extension

    def get_transformed_core_file_name(self, transformation, extension=None):
        to_return = transformation(self.core_file_name)
        if (extension is not None):
            to_return = to_return + extension
        else:
            if (self.extension is not None):
                to_return = to_return + self.extension
        return to_return

    def get_transformed_file_path(self, transformation, extension=None):
        return (self.directory+"/"+
                self.get_transformed_core_file_name(transformation,
                                                    extension=extension))


def format_as_json(jsonable_data):
    return json.dumps(jsonable_data, indent=4, separators=(',', ': '))


class ArgumentToAdd(object):
    """
        Class to append runtime arguments to a string
        to facilitate auto-generation of output file names.
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = (','.join([str(el) for el in self.val])\
                   if (isinstance(self.val, str)==False and
                       hasattr(self.val,"__len__")) else str(self.val))
        return self.argNamePrefix()+string;
        # return self.argNamePrefix()+str(self.val).replace(".","p");


class FloatArgument(ArgumentToAdd):
    """
       Replace the period with a p 
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = str(self.val)
        string = string.replace(".","p")
        return self.argNamePrefix()+string


class BooleanArgument(ArgumentToAdd):

    def transform(self):
        assert self.val  # should be True if you're calling transformation
        return self.argumentName


class CoreFileNameArgument(ArgumentToAdd):

    def transform(self):
        import fileProcessing as fp
        return self.argNamePrefix() + fp.getCoreFileName(self.val)


class ArrArgument(ArgumentToAdd):

    def __init__(self, val, argumentName, sep="+", toStringFunc=str):
        super(ArrArgument, self).__init__(val, argumentName)
        self.sep = sep
        self.toStringFunc = toStringFunc

    def transform(self):
        return self.argNamePrefix() + self.sep.join([self.toStringFunc(x) for x in self.val])


class ArrOfFileNamesArgument(ArrArgument):

    def __init__(self, val, argumentName, sep="+"):
        import fileProcessing as fp
        super(ArrOfFileNamesArgument, self).__init__(val, argumentName,
                                                     sep, toStringFunc=lambda x: fp.getCoreFileName(x))


def addArguments(string, args, joiner="_"):
    """
        args is an array of ArgumentToAdd.
    """
    for arg in args:
        string = string + ("" if arg.val is None or arg.val is False or (hasattr(
            arg.val, "__len__") and len(arg.val) == 0) else joiner + arg.transform())
    return string
class Embedding(object):
    """Represents something that has been embedded in a sequence.
    Think of this as a combination of an embeddable + a start position.
    Arguments:
        what: object representing the thing that has been embedded.\
            Should have`` __str__`` and ``__len__`` defined.\
            Often is an instance of :class:`.AbstractEmbeddable`
        startPos: int, the position relative to the start of the parent\
            sequence at which seq has been embedded
    """

    def __init__(self, what, startPos):
        self.what = what
        self.startPos = startPos

    def __str__(self):
        return "pos-" + str(self.startPos) + "_" + str(self.what)

    @classmethod
    def fromString(cls, string, whatClass=None):
        """Recreate an :class:`.Embedding` object from a string.
        Arguments:
            string: assumed to have format:\
                ``description[-|_]startPos[-|_]whatString``, where
                ``whatString`` will be provided to ``whatClass``
            whatClass: the class (usually a :class:`.AbstractEmbeddable`) that\
                will be used to instantiate the what from the whatString
        Returns:
            The Embedding class called with
            ``what=whatClass.fromString(whatString)`` and
            ``startPos=int(startPos)``
        """
        if (whatClass is None):
            from simdna.synthetic.embeddables import StringEmbeddable
            whatClass = StringEmbeddable
        # was printed out as pos-[startPos]_[what], but the
        #[what] may contain underscores, hence the maxsplit
        # to avoid splitting on them.
        p = re.compile(r"pos\-(\d+)_(.*)$")
        m = p.search(string)
        startPos = m.group(1)
        whatString = m.group(2) 
        return cls(what=whatClass.fromString(whatString),
                   startPos=int(startPos))
      
def getEmbeddingsFromString(string):
    """Get a series of :class:`.Embedding` objects from a string.
    
    Splits the string on commas, and then passes the comma-separated vals
        to :func:`.Embedding.fromString`
    Arguments:
        string: The string to turn into an array of Embedding objects
    Returns:
        an array of :class:`.Embedding` objects
    """
    if len(string) == 0:
        return []
    else:
        embeddingStrings = string.split(",")
        return [Embedding.fromString(x) for x in embeddingStrings]
      
def read_simdata_file(simdata_file, one_hot_encode=False, ids_to_load=None):
    ids = []
    sequences = []
    embeddings = []
    labels = []
    if (ids_to_load is not None):
        ids_to_load = set(ids_to_load)
    def action(inp, line_number):
        if (line_number > 1):
            if (ids_to_load is None or (inp[0] in ids_to_load)):
                ids.append(inp[0]) 
                sequences.append(inp[1])
                embeddings.append(getEmbeddingsFromString(inp[2]))
                labels.append([int(x) for x in inp[3:]])
    perform_action_on_each_line_of_file(
        file_handle=get_file_handle(simdata_file),
        action=action,
        transformation=default_tab_seppd)
    return enum(
            ids=ids,
            sequences=sequences,
            embeddings=embeddings,
            labels=np.array(labels))
  


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
#         line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();

In [10]:
import keras 
import keras_genomics
import numpy as np

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.engine import Layer
from keras.models import Sequential 
import keras.layers as k1
from keras.engine.base_layer import InputSpec

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [11]:
sequences = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").sequences
embeddings = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").embeddings

In [12]:
def check(embeddings): 
    if len(embeddings) > 0:
        all_chars = ""
        for x in embeddings: 
            all_chars += str(x.what)
        if 'TAL' in all_chars and 'GATA' in all_chars: 
            return [1, 1]
        if 'TAL' in all_chars and 'GATA' not in all_chars:
            return [1, 0]
        if 'TAL' not in all_chars and 'GATA' in all_chars:
            return [0, 1]
    return [0,0]

In [13]:
y= np.zeros((0,2))
for x in embeddings: 
    thing = check(x)
    y = np.append(y, [thing], axis=0)

In [14]:
ltrdict = {'a':[1,0,0,0],
           'c':[0,1,0,0],
           'g':[0,0,1,0],
           't':[0,0,0,1],
           'n':[0,0,0,0],
           'A':[1,0,0,0],
           'C':[0,1,0,0],
           'G':[0,0,1,0],
           'T':[0,0,0,1],
           'N':[0,0,0,0]}

def onehot_encode(sequence):
    return np.array([ltrdict[x] for x in sequence])

x = np.array([onehot_encode(x) for x in sequences]) 

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [16]:
x_train.shape

(8000, 1000, 4)

In [17]:
x_train[0].shape

(1000, 4)

In [18]:
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        inputs = inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1]
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))


In [19]:
kernel_size = 15
filters=15
input_length = 1000

In [35]:
seed_num = 3000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History

seed(seed_num)
set_random_seed(seed_num)

In [21]:
from keras.layers import Input
from keras.models import Model
from keras.models import load_model

In [47]:
filename = ('reg_no_batch_standard_%s.txt' % seed_num, str(seed_num))[0]
f = open(filename, 'w')

In [23]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

def evaluate(model, x, y): 
    y_pred = model.predict(x)
    auroc = roc_auc_score(y, model.predict(x)) 
    auprc = average_precision_score(y, model.predict(x))
    print("auroc: " + str(auroc))
    print("auprc: "+ str(auprc))
    f.write("auroc: " + str(auroc) + "\n")
    f.write("auprc: " + str(auprc) + "\n")

# Computing Gradients

In [145]:
import statistics
def get_session():
    try:
        #use the keras session if there is one
        import keras.backend as K
        return K.get_session()
    except:
        #Warning: I haven't really tested this behaviour out...
        global _SESS 
        if _SESS is None:
            print("MAKING A SESSION")
            _SESS = tf.Session()
            _SESS.run(tf.global_variables_initializer()) 
        return _SESS

def compile_func(inputs, outputs):
    if (isinstance(inputs, list)==False):
        print("Wrapping the inputs in a list...")
        inputs = [inputs]
    assert isinstance(inputs, list)
    def func_to_return(inp):
        if len(inp) > len(inputs) and len(inputs)==1:
            print("Wrapping the inputs in a list...")
            inp = [inp]
        assert len(inp)==len(inputs),\
            ("length of provided list should be "
             +str(len(inputs))+" for tensors "+str(inputs)
             +" but got input of length "+str(len(inp)))
        feed_dict = {}
        for input_tensor, input_val in zip(inputs, inp):
            feed_dict[input_tensor] = input_val 
        sess = get_session()
        return sess.run(outputs, feed_dict=feed_dict)  
    return func_to_return

import tensorflow as tf
# grads_tensor = tf.gradients(ys=model.layers[-1].output, xs=model.input)
# grad_func = compile_func(inputs=model.input, outputs=mode.output)
# gradients = grad_func(batch_of_one_hot_encoded_input)

# Reverse Complement

In [37]:
rc_model = keras.models.Sequential()
rc_model.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, 
            input_shape=x_train.shape[1:], padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model.add(k1.core.Activation("relu"))
rc_model.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size, padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model.add(k1.core.Activation("relu"))
rc_model.add(keras_genomics.layers.RevCompConv1D(
            filters=filters, kernel_size=kernel_size,padding="same"))
# rc_model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
rc_model.add(k1.core.Activation("relu"))
rc_model.add(RevCompSumPool())
rc_model.add(k1.pooling.MaxPooling1D(pool_size=20,padding="same", strides=20))
rc_model.add(Flatten())
rc_model.add(keras_genomics.layers.core.Dense(units = 100, activation = "relu"))
rc_model.add(keras_genomics.layers.core.Dense(units = 2))
rc_model.add(k1.core.Activation("sigmoid"))

In [163]:
grads_tensor_rc = tf.gradients(ys = rc_model.layers[-1].output, xs = rc_model.input)
grad_fun = compile_func(inputs = rc_model.input, outputs = grads_tensor_rc)
gradients = grad_fun(x_train[0:500])

Wrapping the inputs in a list...
Wrapping the inputs in a list...


In [164]:
sum = 0
for x in gradients[0]:
    for y in x: 
        for z in y: 
            sum+=z
print(z/2000000)
print("RevComp he_normal")

-2.1142784971743823e-09
RevComp he_normal


In [38]:
rc_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history_rc = rc_model.fit(x_train, y_train, validation_split=0.2,  
                    callbacks=[EarlyStopping(patience=10, restore_best_weights=True),  
                               History()], batch_size=100,  epochs=100) 

Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 6s 901us/step - loss: 0.8096 - acc: 0.5062 - val_loss: 0.6929 - val_acc: 0.5131
Epoch 2/100
6400/6400 [==============================] - 5s 750us/step - loss: 0.6927 - acc: 0.5159 - val_loss: 0.6924 - val_acc: 0.5119
Epoch 3/100
6400/6400 [==============================] - 5s 735us/step - loss: 0.6920 - acc: 0.5202 - val_loss: 0.6927 - val_acc: 0.5112
Epoch 4/100
6400/6400 [==============================] - 5s 756us/step - loss: 0.6904 - acc: 0.5330 - val_loss: 0.6930 - val_acc: 0.4962
Epoch 5/100
6400/6400 [==============================] - 5s 728us/step - loss: 0.6888 - acc: 0.5410 - val_loss: 0.6953 - val_acc: 0.5097
Epoch 6/100
6400/6400 [==============================] - 5s 755us/step - loss: 0.6876 - acc: 0.5430 - val_loss: 0.6955 - val_acc: 0.5025
Epoch 7/100
6400/6400 [==============================] - 5s 737us/step - loss: 0.6845 - acc: 0.5567 - val_loss: 0.6963 - val_acc: 

In [39]:
rc_filename = ('rc_no_batch_standard_%s.h5' % seed_num, str(seed_num))[0]
rc_model.save(rc_filename)
custom_objects = {'RevCompConv1D':keras_genomics.layers.RevCompConv1D, 
                  'RevCompConv1DBatchNorm':keras_genomics.layers.RevCompConv1DBatchNorm,
                  'RevCompSumPool':RevCompSumPool}
rc_model_final = load_model(rc_filename, custom_objects)

In [ ]:
def _compute_fans(shape, data_format='channels_last'):
    """Computes the number of input and output units for a weight shape.
    # Arguments
        shape: Integer shape tuple.
        data_format: Image data format to use for convolution kernels.
            Note that all kernels in Keras are standardized on the
            `channels_last` ordering (even when inputs are set
            to `channels_first`).
    # Returns
        A tuple of scalars, `(fan_in, fan_out)`.
    # Raises
        ValueError: in case of invalid `data_format` argument.
    """
    if len(shape) == 2:
        fan_in = shape[0]
        fan_out = shape[1]
    elif len(shape) in {3, 4, 5}:
        # Assuming convolution kernels (1D, 2D or 3D).
        # TH kernel shape: (depth, input_depth, ...)
        # TF kernel shape: (..., input_depth, depth)
        if data_format == 'channels_first':
            receptive_field_size = np.prod(shape[2:])
            fan_in = shape[1] * receptive_field_size
            fan_out = shape[0] * receptive_field_size
        elif data_format == 'channels_last':
            receptive_field_size = np.prod(shape[:-2])
            fan_in = shape[-2] * receptive_field_size
            fan_out = shape[-1] * receptive_field_size
        else:
            raise ValueError('Invalid data_format: ' + data_format)
    else:
        # No specific assumptions.
        fan_in = np.sqrt(np.prod(shape))
        fan_out = np.sqrt(np.prod(shape))
    return fan_in, fan_out

  
class DebugVarianceScaling(Initializer):
    def __init__(self, scale=1.0,
                 mode='fan_in',
                 distribution='normal',
                 seed=None):
        if scale <= 0.:
            raise ValueError('`scale` must be a positive float. Got:', scale)
        mode = mode.lower()
        if mode not in {'fan_in', 'fan_out', 'fan_avg'}:
            raise ValueError('Invalid `mode` argument: '
                             'expected on of {"fan_in", "fan_out", "fan_avg"} '
                             'but got', mode)
        distribution = distribution.lower()
        if distribution not in {'normal', 'uniform'}:
            raise ValueError('Invalid `distribution` argument: '
                             'expected one of {"normal", "uniform"} '
                             'but got', distribution)
        self.scale = scale
        self.mode = mode
        self.distribution = distribution
        self.seed = seed

    def __call__(self, shape, dtype=None):
        fan_in, fan_out = _compute_fans(shape)
        print("fanin:",fan_in, "fanout:",fan_out, self.scale, self.mode)
        scale = self.scale
        if self.mode == 'fan_in':
            scale /= max(1., fan_in)
        elif self.mode == 'fan_out':
            scale /= max(1., fan_out)
        else:
            scale /= max(1., float(fan_in + fan_out) / 2)
        if self.distribution == 'normal':
            # 0.879... = scipy.stats.truncnorm.std(a=-2, b=2, loc=0., scale=1.)
            stddev = np.sqrt(scale) / .87962566103423978
            return K.truncated_normal(shape, 0., stddev,
                                      dtype=dtype, seed=self.seed)
        else:
            limit = np.sqrt(3. * scale)
            return K.random_uniform(shape, -limit, limit,
                                    dtype=dtype, seed=self.seed)

    def get_config(self):
        return {
            'scale': self.scale,
            'mode': self.mode,
            'distribution': self.distribution,
            'seed': self.seed
        }

DebugVarianceScaling(
    scale=scale,
    mode='fan_avg',
    distribution='uniform',
    seed=None)

# Regular Model

In [44]:
reg_model = keras.models.Sequential()
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=(input_length,4), padding="same"))
# reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        padding="same"))
# reg_model.add(k1.BatchNormalization())
reg_model.add(k1.core.Activation("relu"))
reg_model.add(k1.pooling.MaxPooling1D(pool_size=20,padding="same",
                                               strides=20))
reg_model.add(Flatten())
reg_model.add(k1.Dense(units = 100, activation = "relu"))
reg_model.add(k1.Dense(units = 2))
reg_model.add(k1.core.Activation("sigmoid"))

In [173]:
sum = 0
for x in gradients[0]:
    for y in x: 
        for z in y: 
            sum+=z
print(z/2000000)
print("Reg he_normal")

2.754413988441229e-09
Reg he_normal


In [45]:
reg_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history_reg = reg_model.fit(x_train, y_train, validation_split = 0.2, 
                           callbacks=[EarlyStopping(patience=10,restore_best_weights=True), 
                                      History()], batch_size = 100, epochs = 100)

Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 4s 643us/step - loss: 0.7620 - acc: 0.5080 - val_loss: 0.6994 - val_acc: 0.5122
Epoch 2/100
6400/6400 [==============================] - 3s 412us/step - loss: 0.6844 - acc: 0.5567 - val_loss: 0.6953 - val_acc: 0.5284
Epoch 3/100
6400/6400 [==============================] - 2s 387us/step - loss: 0.6653 - acc: 0.5969 - val_loss: 0.6917 - val_acc: 0.5428
Epoch 4/100
6400/6400 [==============================] - 2s 389us/step - loss: 0.6373 - acc: 0.6371 - val_loss: 0.6899 - val_acc: 0.5609
Epoch 5/100
6400/6400 [==============================] - 2s 387us/step - loss: 0.5748 - acc: 0.6938 - val_loss: 0.6264 - val_acc: 0.6287
Epoch 6/100
6400/6400 [==============================] - 2s 389us/step - loss: 0.5086 - acc: 0.7330 - val_loss: 0.5329 - val_acc: 0.6962
Epoch 7/100
6400/6400 [==============================] - 2s 380us/step - loss: 0.4416 - acc: 0.7805 - val_loss: 0.6313 - val_acc: 

In [46]:
reg_filename = ('reg_no_batch_standard_%s.h5' % seed_num, str(seed_num))[0]
reg_model.save(reg_filename)
reg_model_final = load_model(reg_filename)

# Siamese Architecture

In [72]:
s_model = Sequential([
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
            input_shape=(input_length,4), padding="same"), 
    k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.Conv1D(filters=filters, kernel_size=kernel_size,
              padding="same"), 
    k1.BatchNormalization(), 
    k1.core.Activation("relu"),
    k1.pooling.MaxPooling1D(pool_size=20,padding="same",
                                               strides=20), 
    Flatten(), 
    k1.Dense(units = 100, activation = "relu"),
    k1.Dense(units = 2)
], name = "shared_layers")

main_input = Input(shape=(input_length, 4, ))
rev_input = Input(shape=(input_length, 4, ))

main_output = s_model(main_input)
rev_output = s_model(rev_input)

avg = k1.Average()([main_output, rev_output])
final_out = k1.core.Activation("sigmoid")(avg)
siamese_model = Model(inputs = [main_input, rev_input], outputs = final_out)

merged = keras.layers.concatenate([main_output, rev_output])

predictions = k1.core.Activation("sigmoid")

In [73]:
siamese_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
siamese_model.fit([x_train, x_train[:,::-1,::-1]], y_train, validation_split=0.2,  
                    callbacks=[EarlyStopping(patience=10, restore_best_weights=True), History()],
                    batch_size=100,  epochs=100)

Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 11s 2ms/step - loss: 0.7742 - acc: 0.5016 - val_loss: 0.7029 - val_acc: 0.5056
Epoch 2/100
6400/6400 [==============================] - 6s 925us/step - loss: 0.6882 - acc: 0.5529 - val_loss: 0.7096 - val_acc: 0.5275
Epoch 3/100
6400/6400 [==============================] - 6s 908us/step - loss: 0.6522 - acc: 0.6142 - val_loss: 0.7565 - val_acc: 0.5194
Epoch 4/100
6400/6400 [==============================] - 6s 901us/step - loss: 0.6089 - acc: 0.6608 - val_loss: 1.1923 - val_acc: 0.5175
Epoch 5/100
6400/6400 [==============================] - 6s 892us/step - loss: 0.5369 - acc: 0.7120 - val_loss: 0.6326 - val_acc: 0.6022
Epoch 6/100
6400/6400 [==============================] - 6s 900us/step - loss: 0.4895 - acc: 0.7418 - val_loss: 0.8642 - val_acc: 0.5616
Epoch 7/100
6400/6400 [==============================] - 6s 903us/step - loss: 0.4637 - acc: 0.7559 - val_loss: 1.2914 - val_acc: 0

In [74]:
siamese_filename = ('batch_only_siamese_%s.h5' % seed_num, str(seed_num))[0]
siamese_model.save(siamese_filename)
siamese_model_final = load_model(siamese_filename)

# Evaluating Models

In [40]:
print("Reverse Complement Model")
f.write("Reverse Complement Model\n")
evaluate(rc_model, x_test, y_test)

Reverse Complement Model
auroc: 0.5069699719947236
auprc: 0.5068956563504918


In [48]:
print("Regular Model")
f.write("Regular Model\n")
evaluate(reg_model, x_test, y_test)

Regular Model
auroc: 0.8362035823960353
auprc: 0.8374863949693904


In [77]:
print("Siamese Architecture")
y_pred = siamese_model.predict([x_test, x_test[:,::-1,::-1]])
auroc = roc_auc_score(y_test, y_pred)
auprc = average_precision_score(y_test, y_pred)
print("auroc: " + str(auroc))
print("auprc: "+ str(auprc))
f.write("Siamese Architecture\n")
f.write("auroc: " + str(auroc) + "\n")
f.write("auprc: "+ str(auprc) + "\n")

Siamese Architecture
auroc: 0.8967584015921557
auprc: 0.9126922287627193


26

In [49]:
f.close()